In [ ]:
# Cell 1: Imports and MySQL Initialization with Temporary DF Creation and Cleaning
import os, sys
sys.path.append(os.path.abspath('..'))  # allow importing project root modules

import polars as pl
import pandas as pd
from sqlalchemy import create_engine, text
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from quality_reports import DataQualityReport
from value_reports import ValueDistributionReport

# Connection and base table (using Trino to query MySQL catalog)
conn_str = "trino://root@3.108.199.0:32092/mysql/ap"  # Assuming no password; add :panoiq if needed
base_table_name = "people_data"  # original table

# Create engine
engine = create_engine(conn_str, connect_args={'http_scheme': 'http', 'auth': None})

# Fetch data and create temporary Pandas DF
query = f"SELECT * FROM {base_table_name} LIMIT 1000000"
try:
    df_pandas = pd.read_sql(text(query), engine)
    logging.info(f"Fetched {len(df_pandas)} rows from {base_table_name}")
except Exception as e:
    logging.error(f"Failed to fetch data: {str(e)}")
    raise

# Convert to temporary Polars DF
df = pl.from_pandas(df_pandas)

# Perform cleaning and data handling on the temporary Polars DF
string_cols = [col for col, dtype in df.schema.items() if dtype == pl.Utf8 and col != 'RecordID']
for col in string_cols:
    df = df.with_columns(
        pl.col(col).str.strip_chars().str.replace_all(r'[\n\r\t%]', ' ')
        .str.replace_all(r'\s+', ' ').replace({'': None, 'nan': None, 'None': None, r'\N': None})
    )

# Add RecordID if not present
if 'RecordID' not in df.columns:
    df = df.with_row_index(name='RecordID', offset=1)

logging.info(f"Sanitized temporary Polars DF, columns: {df.columns}")

# Initialize reports
dq_report = DataQualityReport(conn_str, base_table_name)
vd_report = ValueDistributionReport(conn_str, base_table_name, use_altair=True)

# Set the cleaned temporary DF to the report instances
dq_report.df = df
vd_report.df = df

print(f"📊 Reports initialized for table: {base_table_name}")
print(f"🧹 Created and sanitized temporary Polars DF from {base_table_name} (no views)")
print(f"   • Removed newlines (\\n), tabs (\\t), carriage returns (\\r), and % signs (replaced with space)")
print(f"   • Normalized multiple spaces to single space")
print(f"   • Treated empty strings, 'nan', 'None', and '\\N' as NULL")
print(f"📊 Reports will use this cleaned temporary DF for all analysis")

2025-09-09 12:41:22,648 - INFO - Fetched 50578 rows from people_data
2025-09-09 12:41:23,018 - INFO - Sanitized temporary Polars DF, columns: ['RecordID', 'full_name', 'first_and_surname', 'first_name', 'surname', 'dob', 'birth_place', 'postcode_fake', 'gender', 'occupation', 'email', 'phone', 'address', 'city', 'country', 'postal_code']
2025-09-09 12:41:23,020 - INFO - Initialized DataQualityReport for people_data_view
2025-09-09 12:41:23,021 - INFO - Initialized ValueDistributionReport for people_data_view


📊 Reports initialized for table: people_data
🧹 Created and sanitized temporary Polars DF from people_data (no views)
   • Removed newlines (\n), tabs (\t), carriage returns (\r), and % signs (replaced with space)
   • Normalized multiple spaces to single space
   • Treated empty strings, 'nan', 'None' as NULL
   • DOB conversion deferred to report classes
📊 Reports will use this cleaned temporary DF for all analysis


In [2]:
# Cell 2: MySQL Completeness Report (uses the temporary cleaned DF)
completeness_df = dq_report.get_completeness()

alt.VConcatChart(...)

In [3]:
from quality_reports import DataQualityReport

dq_report.create_animated_dashboard(height=800)
print("🎯 Animated dashboard created successfully!")

🎯 Animated dashboard created successfully!


In [4]:

from value_reports import ValueDistributionReport

try:
    results = vd_report.analyze_all_columns(top_n=10, show_individual_plots=True)
    print("📊 Value Distribution Analysis Complete!")
    print(f"✅ Analyzed {len(results)} columns with clean, sanitized data")
except Exception as e:
    print(f"Error during analysis: {str(e)}")
    import traceback
    traceback.print_exc()  # Print full stack trace for debugging

2025-09-09 12:41:23,710 - INFO - Creating visualization for full_name, type: categorical
2025-09-09 12:41:23,712 - INFO - Preparing categorical data for full_name, dtype: String
2025-09-09 12:41:23,768 - INFO - Creating categorical chart for full_name, data columns: ['full_name', 'Count']


Processing column: full_name, type: categorical
vc columns after rename: ['value', 'Count']


alt.LayerChart(...)

2025-09-09 12:41:23,865 - INFO - Creating visualization for first_and_surname, type: categorical
2025-09-09 12:41:23,870 - INFO - Preparing categorical data for first_and_surname, dtype: String


Processing column: first_and_surname, type: categorical
vc columns after rename: ['value', 'Count']


...

Processing column: postal_code, type: categorical
vc columns after rename: ['value', 'Count']


alt.LayerChart(...)

2025-09-09 12:41:26,391 - INFO - Preparing categorical data for full_name, dtype: String
2025-09-09 12:41:26,427 - INFO - Creating categorical chart for full_name, data columns: ['full_name', 'Count']
2025-09-09 12:41:26,500 - INFO - Preparing categorical data for first_and_surname, dtype: String
2025-09-09 12:41:26,540 - INFO - Creating categorical chart for first_and_surname, data columns: ['first_and_surname', 'Count']
2025-09-09 12:41:26,593 - INFO - Preparing categorical data for first_name, dtype: String
2025-09-09 12:41:26,612 - INFO - Creating categorical chart for first_name, data columns: ['first_name', 'Count']
2025-09-09 12:41:26,673 - INFO - Preparing categorical data for surname, dtype: String
2025-09-09 12:41:26,689 - INFO - Creating categorical chart for surname, data columns: ['surname', 'Count']
2025-09-09 12:41:26,741 - INFO - Creating histogram for dob, x_label: dob, dtype: Date
2025-09-09 12:41:26,744 - INFO - Preparing categorical data for birth_place, dtype: Stri

alt.VConcatChart(...)

📊 Value Distribution Analysis Complete!
✅ Analyzed 15 columns with clean, sanitized data
